# Learning goals
After this week's lesson you should be able to:
- Refresher on data structure (DF vs Series vs List vs Array)
- Checking a columns data types and converting types
- Rename a (geo)dataframe column 
- parsing dates
- slicing strings
- handling missing data. 
    - filtering out missing data
    - replacing missing data with the mean
- Merging 
- More on groupby-and-summarize
- (Defining and using a function)
- (iterating over rows)
- (applying a function)

This week's lessons are adapted from:
- [PPD599: Advanced Urban Analytics](https://github.com/gboeing/ppd599/tree/main/syllabus)
- [Geo-Python Lesson 5](https://geo-python-site.readthedocs.io/en/latest/notebooks/L5/processing-data-with-pandas.html)

In [1]:
# We are going to start importing the libraries we need
# all in one cell. 
# It is a good practice to keep all the imports in one cell so that
# we can easily see what libraries we are using in the notebook.
import pandas as pd
import numpy as np
import geopandas as gpd

/var/folders/c5/dps3lj6n2sz8zkk_381lj9b1hnpv4f/T/ipykernel_85612/3517648564.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


# 0. Refresher
Before we move on, let's go over the different types of data structures we've encountered so far. We are going to cover: 
- Pandas DataFrames
- Pandas Series
- Lists
- Arrays

## 0.1 Pandas: Dataframes

In [2]:
msa = pd.read_csv('msa_by_pop.csv')

What makes `msa` a dataframe? 

In [3]:
msa.head()

,Rank,MSA,population_2021_est,population_2020,perc_change
0,1,"New York-Newark-Jersey City, NY-NJ-CT-PA MSA",19768458,20140470,1.85%
1,2,"Los Angeles-Long Beach-Anaheim, CA MSA",12997353,13200998,1.54%
2,3,"Chicago-Naperville-Elgin, IL-IN-WI MSA",9509934,9618502,1.13%
3,4,"Dallas-Fort Worth-Arlington, TX MSA",7759615,7637387,+1.60%
4,5,"Houston-The Woodlands-Sugar Land, TX MSA",7206841,7122240,+1.19%


In [4]:
# We can use the type() function to see what type of object we have
type(msa)

pandas.core.frame.DataFrame

Now let's select just the `population_2020` column from `msa`. What makes the following a pandas Series? 

## 0.2 Pandas: Series

In [5]:
msa['population_2020'].head()

0    20140470
1    13200998
2     9618502
3     7637387
4     7122240
Name: population_2020, dtype: int64

In [6]:
type(msa['population_2020'])

pandas.core.series.Series

In [7]:
population = pd.Series([19768458, 12997353, 9509934, 7759615, 7206841],    
            index=['NYC', 'LA', 'Chicago', 'Dallas', 'Houston'])
population


NYC        19768458
LA         12997353
Chicago     9509934
Dallas      7759615
Houston     7206841
dtype: int64

In [8]:
type(population)

pandas.core.series.Series

## 0.3 Python data structures: Lists

Now, let's just select the values from `population` as a **list**. 

In [9]:
## This is a list of numbers
## A list is a collection of objects of any type.
## It is created by putting the objects in square brackets

list1 = [19768458, 12997353, 9509934, 7759615, 7206841]

In [10]:
type(list1)

list

In [11]:
list.mean()

AttributeError: type object 'list' has no attribute 'mean'

In [12]:
## This is also a list
## But instead of being a list of numbers, it is a list of strings

list2 = ['NYC', 'LA', 'Chicago', 'Dallas', 'Houston']

## 0.4 Numpy: Arrays

We are not going to cover numpy directly in the class. But there is another list-like data structure used by the `numpy` library called an **array**. 

Though we are inputting a list above to create a pandas Series, pandas will turn this list into a **numpy array**. All pandas series are basically just generalized version of numpy arrays. 

An array is a collection of objects of the same type. It is created by putting the objects in square brackets and using the `np.array()` function. A numpy array is most used for numerical calculations such as finding the mean, min, sum of a set of values. 

In [13]:
list1_as_array = np.array(list1)

In [14]:
list1_as_array

array([19768458, 12997353,  9509934,  7759615,  7206841])

In [15]:
list1_as_array.mean()

11448440.2

# 1. Data cleaning
As you might have already seen, when we work with data, it is not always in a shape that we can use it, sometimes column names are misspelled, there are missing values. You may also have noticed that often we can extract information from columns that might make them easier to work with. All these steps can be considered part of a data cleaning or data wrangling process, where we get the dataset ready to be used more effectively for our analysis purposes. 




## 1.1 Getting the data
Let's say we want to compare the relationship between the **total number of students in a general ed public school** to the **money spent on new school construction and improvements in that school**. 

### School Construction Authority

First, go ahead and download the [Active Projects Under Construction](https://data.cityofnewyork.us/Housing-Development/Active-Projects-Under-Construction/8586-3zfm) dataset as a CSV and save it down to the folder where this notebook is. This is a dataset of new school projects (Capacity) and Capital Improvement Projects (CIP) currently under Construction, created by the School Construction Authority. 




In [17]:
## Here we are going to read a csv directly from the web
## We are going to use the read_csv() function from the pandas library
## 

projects_under_const = pd.read_csv('Active_Projects_Under_Construction.csv')

Also, go ahead and download the data dictionary `SCA Active Projects in Construction Data Dictionary.xlsx`. Data dictionaries often have explanations for what each column name represents and other useful information about the data. 


If you open up the data dictionary, does it correspond to the "Columns in this Dataset" section in the NYC OpenData's page on this dataset? No, right? We have to be careful about these inconsistencies, even in official portals.

Taking a look at the first five rows we can already see there is a lot of missing data in this dataset. 

In [18]:
projects_under_const.head()

,School Name,BoroughCode,Geographical District,Project Description,Construction Award,Project type,Building ID,Building Address,City,Postcode,...,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location 1,Data As Of
0,NaN,M,2,NaN,0.0,CAP,M777,227 WEST 27TH STREET,Manhattan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAYSIDE HIGH SCHOOL - QUEENS,Q,26,FY19 RESO A AUDITORIUM UPGRADE,1261000.0,CIP,Q405,32-24 CORPORAL KENNEDY STREET,Queens,10301.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01/06/2022
2,P.S. @ PARCEL F - QUEENS,Q,30,Demo,0.0,CAP,Q375,2ND STREET BETWEEN 56TH AND 57TH AVENUE,Queens,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/30/2018
3,3K CENTER @ 3893 DYRE AVENUE - BRONX,X,11,Lease,6262000.0,CAP,X501,3893 DYRE AVEUNE,Bronx,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/04/2022
4,P.S. 129 - QUEENS,Q,25,Addition,0.0,CAP,Q129,128-02 7TH AVENUE,Queens,11356.0,...,40.790638,-73.839771,7.0,19.0,945.0,4096774.0,4.039760e+09,Whitestone,"(40.790638, -73.839771)",02/06/2019


### Class size dataset
Also download the [2021 - 2022 Average Class Size by School](https://data.cityofnewyork.us/Education/2021-2022-Average-Class-Size-by-School/sgr7-hhwp) dataset, along with it's attachments. (Here, only `2021-2022 Average Class Size By School DD.xlsx` is the data dictoinary, the other is the dataset as an excel spreadsheet). 


In [20]:
class_size = pd.read_csv('2021_-_2022_Average_Class_Size_by_School.csv')

In [21]:
class_size.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,13,1,13.0,<15,<15
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,17,1,17.0,17,17
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,8,1,8.0,<15,<15
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,18,1,18.0,18,18
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,8,1,8.0,<15,<15


Here, most of the columns make sense to me. From the data dictionary, I can see that Program Type is coded as follows:

- General Education (Gen Ed), 
- Integrated Co-Teaching (ICT), 
- Gifted and Talented (G&T), 
- Self-Contained (SC)
- Accelerated (Acc)"


What does not make sense is the `Minimum Class Size` column, which seems to be the same as the maximum class size column in some cases. Therefore, I'll likely not use this column.

## 1.2 Assessing Data Types
One of the next things we'll check is the data type for each column to make sure that they are in the right format. 

In [22]:
class_size.dtypes

DBN                    object
School Name            object
Grade Level            object
Program Type           object
Number of Students      int64
Number of Classes       int64
Average Class Size    float64
Minimum Class Size     object
Maximum Class Size     object
dtype: object

I would not necessarily change the data types for all columns (especially when there are a lot), but just the ones that you might potentially need. Here, `Maximum Class Size` is an `object` format (I'm going to ignore `Minimum Class Size` for now), likely because the size is sometimes input as `<INT` and sometimes `INT`. 



## 1.3 Removing strings

In order to change the data type of the min and max class size to an `int` we have to clean up those columns a little bit. 

The function `.replace('str_to_be_replaced','str_to_replace_with)` will take `str_to_be_replaced` and replace it with `str_to_replace_with`. Here, I'm setting `<` to be replaced by nothing which is expressed as an empty string `""`. 

In [23]:
# Again, I'm going to check that the function worked as expected first
# here .str is a method that is applied to a string
# it is a vectorized string method
class_size['Maximum Class Size'].str.replace('<', "")

# (Vectorization is the process of converting an algorithm from operating on a single value at a time to operating on a set of values at one time
# but it's not super important for us to know what this is right now)

0        15
1        17
2        15
3        18
4        15
         ..
12440    12
12441    10
12442    12
12443    12
12444    12
Name: Maximum Class Size, Length: 12445, dtype: object

Now lets assign the result to a new column and why not rename the column to something in snake case. 


In [24]:
class_size['max_class_size_clean'] = class_size['Maximum Class Size'].str.replace('<', "")

In [25]:
class_size.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size,max_class_size_clean
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,13,1,13.0,<15,<15,15
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,17,1,17.0,17,17,17
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,8,1,8.0,<15,<15,15
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,18,1,18.0,18,18,18
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,8,1,8.0,<15,<15,15


Now let's see if we can turn the column `min_class_size_clean` into an `integer`. 

In [26]:
class_size['max_class_size_clean'].astype(int)

ValueError: invalid literal for int() with base 10: '>34'

Oops, I guess we also have to replace the greater than `>`. 

In [27]:
class_size['max_class_size_clean'] = class_size['max_class_size_clean'].str.replace('>', "")

## 1.4 Changing data types
Now let's try to change the data type for `max_class_size_clean`. 

`.astype()` changes your column types for a particular column. 


In [28]:
## What I've done here is replace the old `max_class_size_clean` column with 
## a version of it that is an int
class_size['max_class_size_clean'] = class_size['max_class_size_clean'].astype(int)

In [29]:
# Notice that `int` from above defaults to 64 bit integers. 
class_size['max_class_size_clean'].dtype

dtype('int64')

## 1.5 Slicing strings

### 1.5.1
The `projects_under_const` has a `Data as Of` column, which gives us some temporal variation in when, at least the data was added to the table. It could be useful, for instance, if we think that `Data as Of` is a rough proxy for when the project was funded or approved. 

In [30]:
# Remember that NaN means "Not a Number".
# In other words, it is a missing value
projects_under_const['Data As Of'].head()

0           NaN
1    01/06/2022
2    10/30/2018
3    08/04/2022
4    02/06/2019
Name: Data As Of, dtype: object

Let's say we want to extract year from these dates. We have another string-related function we can apply to all of our values under `Data As Of`. 

`.split()` splits strings around given separator/delimiter to create a list of strings. 

Here, we will use `/` as our separator. 

In [31]:
projects_under_const['Data As Of'].str.split('/')

0                  NaN
1       [01, 06, 2022]
2       [10, 30, 2018]
3       [08, 04, 2022]
4       [02, 06, 2019]
             ...      
8996    [11, 02, 2022]
8997    [11, 02, 2022]
8998    [11, 02, 2022]
8999    [11, 02, 2022]
9000    [11, 02, 2022]
Name: Data As Of, Length: 9001, dtype: object

Now we just have to get the last value (where it exists) and create a new column with the year. 

In [32]:
## [-1] is a way to access the last element of a list
projects_under_const['data_year'] = projects_under_const['Data As Of'].str.split('/').str[-1]

In [33]:
# Notice that when there was an NaN, the split function returned a NaN
projects_under_const['data_year'].head()

0     NaN
1    2022
2    2018
3    2022
4    2019
Name: data_year, dtype: object

### 1.5.2
We will eventually be comparing school attendance characteristics to money allocated through **merging along a common column name** at the **school level**.

What are out options here? Let's take  look. 

In [34]:
projects_under_const.head()

,School Name,BoroughCode,Geographical District,Project Description,Construction Award,Project type,Building ID,Building Address,City,Postcode,...,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location 1,Data As Of,data_year
0,NaN,M,2,NaN,0.0,CAP,M777,227 WEST 27TH STREET,Manhattan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAYSIDE HIGH SCHOOL - QUEENS,Q,26,FY19 RESO A AUDITORIUM UPGRADE,1261000.0,CIP,Q405,32-24 CORPORAL KENNEDY STREET,Queens,10301.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01/06/2022,2022
2,P.S. @ PARCEL F - QUEENS,Q,30,Demo,0.0,CAP,Q375,2ND STREET BETWEEN 56TH AND 57TH AVENUE,Queens,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/30/2018,2018
3,3K CENTER @ 3893 DYRE AVENUE - BRONX,X,11,Lease,6262000.0,CAP,X501,3893 DYRE AVEUNE,Bronx,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08/04/2022,2022
4,P.S. 129 - QUEENS,Q,25,Addition,0.0,CAP,Q129,128-02 7TH AVENUE,Queens,11356.0,...,-73.839771,7.0,19.0,945.0,4096774.0,4.039760e+09,Whitestone,"(40.790638, -73.839771)",02/06/2019,2019


In [35]:
class_size.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size,max_class_size_clean
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,13,1,13.0,<15,<15,15
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,17,1,17.0,17,17,17
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,8,1,8.0,<15,<15,15
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,18,1,18.0,18,18,18
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,8,1,8.0,<15,<15,15


Even though there is a **School Name** column in both datasets, the format seems to be quite different. 
- For the `projects_under_const` dataset, the school names are all over the place. Some are the name and borough separated by a `-`, some also include an `@` followed by a rough locationn. 
- For the `class_size` df, the school names are consistent, but we can see that it might be a pain to match the two. 

In [36]:
projects_under_const['School Name']

0                                        NaN
1               BAYSIDE HIGH SCHOOL - QUEENS
2                   P.S. @ PARCEL F - QUEENS
3       3K CENTER @ 3893 DYRE AVENUE - BRONX
4                          P.S. 129 - QUEENS
                        ...                 
8996                     P.S. 236 - BROOKLYN
8997                        P.S. 277 - BRONX
8998                       P.S. 5 - BROOKLYN
8999                        P.S. 182 - BRONX
9000                        I.S. 127 - BRONX
Name: School Name, Length: 9001, dtype: object

In [37]:
class_size['School Name']

0                              PS 015 ROBERTO CLEMENTE
1                              PS 015 ROBERTO CLEMENTE
2                              PS 015 ROBERTO CLEMENTE
3                              PS 015 ROBERTO CLEMENTE
4                              PS 015 ROBERTO CLEMENTE
                             ...                      
12440                  PS 377 ALEJANDRINA B DE GAUTIER
12441                        JHS 383 PHILIPPA SCHUYLER
12442                        JHS 383 PHILIPPA SCHUYLER
12443                      PS /IS 384 FRANCES E CARTER
12444    EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION
Name: School Name, Length: 12445, dtype: object

Instead, I noticed that there's a `Building ID` column in the `projects_under_constr` DF (dataframe, for short) that, though is described unhelpfully as "ID of the Building" in the documentation, looks to be similar to the `DBN` from `class_size` DF. In fact, when I look at what `DBN` is in the class size documentation, it says that this column "Denotes cocatenation[sic] of district, borough and three digit school number."

I'm going to guess here that if I extract the "borough and three digit school number" part of `DBN`, this will match my `Building ID` column. 

Thankfully, it seems like there is a fixed number of characters I need extract from `DBN`: 
- Borough = 1
- School number = 3

In total, I will need the last 4 characters from `DBN`. We'll do this again with a string splice. 

In [38]:
# Here I am going to use the str method to get the last 4 characters of the DBN
# within the square brackets, I am taking everything fourth from the end onwards
# That's what -4 means

class_size['DBN'].str[-4:]

0        M015
1        M015
2        M015
3        M015
4        M015
         ... 
12440    K377
12441    K383
12442    K383
12443    K384
12444    K562
Name: DBN, Length: 12445, dtype: object

Quick review of selecting ranges:

In [39]:
# It's a little strange because backwards counting starts at -1

class_size['DBN'].str[-1:]

0        5
1        5
2        5
3        5
4        5
        ..
12440    7
12441    3
12442    3
12443    4
12444    2
Name: DBN, Length: 12445, dtype: object

In [40]:
## Here, 4: means that I want to start at the fifth character 
## because python starts counting at 0 for forward counting
class_size['DBN'].str[4:]


0        15
1        15
2        15
3        15
4        15
         ..
12440    77
12441    83
12442    83
12443    84
12444    62
Name: DBN, Length: 12445, dtype: object

In [41]:
## And if I wanted to select a slice of the string in the middle
## I can do the following
class_size['DBN'].str[1:4]

0        1M0
1        1M0
2        1M0
3        1M0
4        1M0
        ... 
12440    2K3
12441    2K3
12442    2K3
12443    2K3
12444    2K5
Name: DBN, Length: 12445, dtype: object

Back to our exericse, let's assign our slice to a new colunn called `bid`

In [42]:
class_size['bid'] = class_size['DBN'].str[-4:]

In [43]:
class_size.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size,max_class_size_clean,bid
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,13,1,13.0,<15,<15,15,M015
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,17,1,17.0,17,17,17,M015
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,8,1,8.0,<15,<15,15,M015
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,18,1,18.0,18,18,18,M015
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,8,1,8.0,<15,<15,15,M015


## 1.6 Handling missing data
Now, let's say that our analysis depends knowing the year the data was created. There are a few ways of handling missing data. 

### 1.6.1 Removing rows 
We can remove those rows with data missing from a column that we are planning to use in our analysis. 

In [44]:
# Here we are going to use the isna() function to check if the data_year column has a NaN
# isna() returns a boolean (True or False) for each row
# and we are going to use that boolean to filter the dataframe. 
# We are going to keep only the rows where the data_year column is not a NaN

projects_under_const_new = projects_under_const[projects_under_const['data_year'].isna()==False]

### 1.6.2 Replacing missing data
We can also replace the missing data with certain values: 
- We can replace the data with the mean of the non-NaN column values, for numerical values. (For instance, if our columns were something like "adult heights", then replacing the NaN with the mean values in the columns would allow us to leave the sample mean unchanged, which might be good for regression purposes). 
- We can also replace with the median (if you think there are outliers in the sample that might be skewing the mean)
- Replacing with the mode (most frequent value) would make more sense if we think that there's some default value 

**What would you do here?**

In [46]:
# This gets the mode of the data_year column
mode_year = projects_under_const['data_year'].mode()


In [47]:
# This fills the NaNs with the mode using the fillna() function
# fillna() is a method that fills in missing values with a value of your choice
projects_under_const['data_year'].fillna(mode_year)


0       2022
1       2022
2       2018
3       2022
4       2019
        ... 
8996    2022
8997    2022
8998    2022
8999    2022
9000    2022
Name: data_year, Length: 9001, dtype: object

In [73]:
# Now write over the old data_year column with the new one
projects_under_const['data_year'] = projects_under_const['data_year'].fillna(mode_year)

## Q1. In-Class Exercise 1 (5 pts)
In the end, was it the best idea to replace the NaN data in `data_year`? Why or why not? 

### Response
**I don't think it was the best idea to replace NaN values in data_year**, because this leads to misleading info about the distribution of years in the dataset. If you were to do any temporal analysis with this dataframe, the results would probably be skewed towards the mode (2022), which isn't necessarily a true representation of the empirical distribution. If we gained more info on the true distributions of the years, we may have a better sense of how best to replace NaN data, but currently it's best to leave it as is or omit entirely. 

## 1.7 Aggregating data: A review of groupby-and-summarize
Last week, we introduced the "groupby-and-summarize" operation that is very common in pandas. It's common because we often want to aggregate data by some category. For example, we might want to know the total amount of construction money allocated by school. Or we might want to know the total number of students in each school.

For the projects under construction, let's group by the `Building ID`, which we had a hunch was the same as the `DBN` (Borough and School ID) to get the: 
- Total construction award amount per school

In [49]:
## Remember that .sum() will only sum the numeric columns
projects_under_const.groupby('Building ID').sum()

/var/folders/c5/dps3lj6n2sz8zkk_381lj9b1hnpv4f/T/ipykernel_85612/2229213777.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  projects_under_const.groupby('Building ID').sum()


,Geographical District,Construction Award,Postcode,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL
Building ID,,,,,,,,,,
K001,165,4.409425e+06,108098.0,406.490420,-740.122890,2470.0,349.0,800.0,30118210.0,3.007550e+10
K002,153,6.349880e+07,85532.0,325.249464,-591.612664,2472.0,320.0,6416.0,27025608.0,2.440400e+10
K003,52,2.693780e+07,43962.0,162.730652,-295.821352,912.0,144.0,908.0,12229624.0,1.207992e+10
K005,64,1.700470e+07,43238.0,162.742780,-295.689852,1212.0,164.0,1508.0,12159624.0,1.205960e+10
K007,171,2.768991e+06,74410.0,284.827879,-517.111763,2135.0,259.0,827407.0,21636986.0,2.128742e+10
...,...,...,...,...,...,...,...,...,...,...
X843,170,1.272268e+08,156664.0,612.983895,-1108.421370,2105.0,203.0,6015.0,30207450.0,3.047520e+10
X862,27,3.513000e+06,31368.0,122.492985,-221.708115,9.0,35.0,447.0,6012906.0,6.078450e+09
X930,50,8.898153e+07,41540.0,163.428268,-295.595744,820.0,60.0,153204.0,8054244.0,8.126000e+09


Most of these columns are gibberish after we sum (for ex: we don't need a sum of latitudes and longitudes by school). Let's just select the columns we want to use: 

In [50]:
# Remeber the brackets after a DF allow you to select columns
projects_under_const.groupby('Building ID').sum()['Construction Award']

/var/folders/c5/dps3lj6n2sz8zkk_381lj9b1hnpv4f/T/ipykernel_85612/1098362502.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  projects_under_const.groupby('Building ID').sum()['Construction Award']


Building ID
K001    4.409425e+06
K002    6.349880e+07
K003    2.693780e+07
K005    1.700470e+07
K007    2.768991e+06
            ...     
X843    1.272268e+08
X862    3.513000e+06
X930    8.898153e+07
X970    9.079000e+06
X973    3.092000e+05
Name: Construction Award, Length: 1181, dtype: float64

Let's assign this to a new variable name. 

In [51]:
projects_under_const_agg = projects_under_const.groupby('Building ID').sum()['Construction Award']

/var/folders/c5/dps3lj6n2sz8zkk_381lj9b1hnpv4f/T/ipykernel_85612/10828810.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  projects_under_const_agg = projects_under_const.groupby('Building ID').sum()['Construction Award']


Here you can see that the result is a **pandas Series**. To make this easier to work with during the merge, let's transform this into a pandas DF. 

I'm going to use a function call `.reset_index()` as a trick to do this. `.reset_index()` is a method that resets the index of a dataframe to a column of your choice. The default is to reset the index to a column of sequential numbers

In [52]:
# See how Building ID, which was the index before, is now a column. 
# and the index is i just 0,...,1180

projects_under_const_agg.reset_index()

,Building ID,Construction Award
0,K001,4.409425e+06
1,K002,6.349880e+07
2,K003,2.693780e+07
3,K005,1.700470e+07
4,K007,2.768991e+06
...,...,...
1176,X843,1.272268e+08
1177,X862,3.513000e+06
1178,X930,8.898153e+07
1179,X970,9.079000e+06


In [53]:
projects_under_const_agg = projects_under_const_agg.reset_index()

In [54]:
projects_under_const_agg

,Building ID,Construction Award
0,K001,4.409425e+06
1,K002,6.349880e+07
2,K003,2.693780e+07
3,K005,1.700470e+07
4,K007,2.768991e+06
...,...,...
1176,X843,1.272268e+08
1177,X862,3.513000e+06
1178,X930,8.898153e+07
1179,X970,9.079000e+06


Let's do something similar with the `class_size` df. As we can see from the below, our data is likely one row per grade and program. We want to aggregate this to the school level. 

In [55]:
class_size.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size,max_class_size_clean,bid
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,13,1,13.0,<15,<15,15,M015
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,17,1,17.0,17,17,17,M015
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,8,1,8.0,<15,<15,15,M015
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,18,1,18.0,18,18,18,M015
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,8,1,8.0,<15,<15,15,M015


I'm first going to filter my DF since I just want 'Gen Ed' in order not to skew the representative class size by special programs. 

In [56]:
# .unique() returns a list of all the unique values in a column
class_size['Program Type'].unique()

array(['G&T', 'ICT', 'Gen Ed', 'ICT & G&T', 'SC 12:1:1', 'SC 12:1',
       'SC 8:1:1', 'SC 12:1:4', 'SC 6:1:1', 'SC 15:1'], dtype=object)

In [57]:
# I am going to use the == operator to check if the value in the Program Type column is equal to 'Gen Ed'
# Then we'll set this filtered dataframe to a new variable
# and use that new dataframe from now on. 
class_size_new = class_size[class_size['Program Type']=='Gen Ed']

In [58]:
class_size_new.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size,max_class_size_clean,bid
18,01M020,PS 020 ANNA SILVER,K,Gen Ed,33,2,16.5,<15,20,20,M020
20,01M020,PS 020 ANNA SILVER,1,Gen Ed,31,2,15.5,15,16,16,M020
22,01M020,PS 020 ANNA SILVER,2,Gen Ed,24,2,12.0,<15,15,15,M020
24,01M020,PS 020 ANNA SILVER,3,Gen Ed,30,3,10.0,<15,<15,15,M020
26,01M020,PS 020 ANNA SILVER,4,Gen Ed,18,2,9.0,<15,<15,15,M020


Now, to get the total number of students in each school, I'll have to: 
- Multiply `Number of Classes` and `Number of Students` (let's assume this is per class)
- Sum the total number of students across all classes in a school. 

In [59]:
class_size_new['total_students_in_grade'] = class_size_new['Number of Students'] * class_size_new['Number of Classes']
# Yes, ignore the SettingWithCopyWarning. 

/var/folders/c5/dps3lj6n2sz8zkk_381lj9b1hnpv4f/T/ipykernel_85612/4113767118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_size_new['total_students_in_grade'] = class_size_new['Number of Students'] * class_size_new['Number of Classes']


In [60]:
class_size_new.head()

,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size,max_class_size_clean,bid,total_students_in_grade
18,01M020,PS 020 ANNA SILVER,K,Gen Ed,33,2,16.5,<15,20,20,M020,66
20,01M020,PS 020 ANNA SILVER,1,Gen Ed,31,2,15.5,15,16,16,M020,62
22,01M020,PS 020 ANNA SILVER,2,Gen Ed,24,2,12.0,<15,15,15,M020,48
24,01M020,PS 020 ANNA SILVER,3,Gen Ed,30,3,10.0,<15,<15,15,M020,90
26,01M020,PS 020 ANNA SILVER,4,Gen Ed,18,2,9.0,<15,<15,15,M020,36


Now let's groupby `bid` and sum all the grades within each school. 

In [61]:
class_size_new.groupby('bid').sum()

/var/folders/c5/dps3lj6n2sz8zkk_381lj9b1hnpv4f/T/ipykernel_85612/635825655.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  class_size_new.groupby('bid').sum()


,Number of Students,Number of Classes,Average Class Size,max_class_size_clean,total_students_in_grade
bid,,,,,
K001,492,25,117.4,131,2121
K002,226,10,66.7,75,783
K003,201,12,100.5,105,402
K005,70,4,70.0,75,70
K006,347,19,111.1,122,1169
...,...,...,...,...,...
X584,118,6,59.0,65,236
X593,63,3,63.0,63,63
X594,104,5,60.0,62,192


Again, we'll just need the `total_students_in_grade` column here. And I'm going to do the `reset_index()` trick again. This time, I'm going to string all these steps together

In [62]:
# Pandas reads this code from left to right and will apply each function on the right to the everything on the left
# So, first we are going to group by bid
# Then we are going to sum each group
# Then from the entire summed dataframe, we are going to select the total_students_in_grade column
# Selecting that series, we are going to reset the index to create our new dataframe. .

class_size_new_agg = class_size_new.groupby('bid').sum()['total_students_in_grade'].reset_index()

/var/folders/c5/dps3lj6n2sz8zkk_381lj9b1hnpv4f/T/ipykernel_85612/597514235.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  class_size_new_agg = class_size_new.groupby('bid').sum()['total_students_in_grade'].reset_index()


In [63]:
class_size_new_agg.head()

,bid,total_students_in_grade
0,K001,2121
1,K002,783
2,K003,402
3,K005,70
4,K006,1169


Finally, we get to do our merge. We are going to merge 
- `projects_under_cont_agg`
- `class_size_new_agg`

In [64]:
class_size_new_agg.merge(projects_under_const_agg,left_on='bid',right_on='Building ID', how='left')

,bid,total_students_in_grade,Building ID,Construction Award
0,K001,2121,K001,4409425.0
1,K002,783,K002,63498798.0
2,K003,402,K003,26937800.0
3,K005,70,K005,17004700.0
4,K006,1169,NaN,NaN
...,...,...,...,...
1053,X584,236,NaN,NaN
1054,X593,63,NaN,NaN
1055,X594,192,NaN,NaN
1056,X595,111,NaN,NaN


In [65]:
merged_df = class_size_new_agg.merge(projects_under_const_agg,left_on='bid',right_on='Building ID', how='left')

Ok, finally, to get to our answer, we're going to apply the `.corr()` function to our dataframe. The [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) tells us that this function computes pairwise correlation of columns, excluding NA/null values.

The default method is a 'Pearson' correlation, with all methods being: 
- pearson : standard correlation coefficient
- kendall : Kendall Tau correlation coefficient
- spearman : Spearman rank correlation

In [66]:
# Yikes, 0.079 correlation. I guess I assumed wrong that there would be an strong correlation between the number of students in a school and the amount of money spent on construction.
merged_df.corr()

/var/folders/c5/dps3lj6n2sz8zkk_381lj9b1hnpv4f/T/ipykernel_85612/2175632114.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  merged_df.corr()


,total_students_in_grade,Construction Award
total_students_in_grade,1.000000,0.078964
Construction Award,0.078964,1.000000


In [72]:
# Using a rank correlation instead of a linear correlation hasn't improved the correlation much.
# (Don't worry about what this is right now.)
merged_df.corr(method = 'spearman')

/var/folders/c5/dps3lj6n2sz8zkk_381lj9b1hnpv4f/T/ipykernel_85612/1569521364.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  merged_df.corr(method = 'spearman')


,total_students_in_grade,Construction Award
total_students_in_grade,1.000000,0.151567
Construction Award,0.151567,1.000000


This was a long lecture, no more exercises!